In [1]:
import glob
from os.path import join, basename, splitext, abspath

In [2]:
def readFrameAnnotation(annotationFile):
    """
        read annotation file
        return the list of annotation ([start, end], gesture)
    """
    anno = []
    for l in open(annotationFile).read().splitlines():
        s = l.split(' ')
        anno += [ ([int(s[1]), int(s[2])], int(s[0])-1)]
    return anno

In [3]:
class Annotation:
    def __init__(self, annotFile=None):
        self.seqs = []
        self.gesture = []
        
        if not annotFile is None:
            for l in open(annotFile).read().splitlines():
                s = l.split(' ')
                self.seqs.append( (int(s[1]), int(s[2])) )
                self.gesture.append(int(s[0])-1)
    
    def addAnnotation(self, annotFile):
        for l in open(annotFile).read().splitlines():
            s = l.split(' ')
            self.seqs.append(int(s[1]), int(s[2]))
            self.gesture.append(int(s[0])-1)
            
    def __len__(self):
        return len(self.gesture)
    
    def __getitem__(self, i):
        return self.seqs[i], self.gesture[i]
    
    def __str__(self):
        s = "Annotation("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i])
        else :
            s += str(self[0]) + ' ... ' + str(self[-1])
        return s +")"
    
    def __repr__(self):
        s = "Annotation("
        for i in range(len(self)):
            s += str(self[i])
        return s +")"

In [4]:
class VideoDataset:
    def __init__(self, folderName):
        """
            Look for all .mp4 files in the folder
            The folder must have a annotation folder inside

            return a list of [ (video path, annotation) ]
        """
        p = join(folderName, "*.mp4")
        annoPath = join(folderName, "annotation")
        
        self.annotations = []
        self.videoList = []
        self.folderName = folderName
        
        for v in glob.glob(p):
            bname = basename(v)
            annoFile = join(annoPath, splitext(bname)[0])
            anno = Annotation(annoFile)
            self.annotations.append(anno)
            self.videoList.append(abspath(v)) #to avoid 2 identical files
            
    def __len__(self):
        return len(self.videoList)
    
    def __getitem__(self, i):
        return self.videoList[i], self.annotations[i]
    
    def __repr__(self):
        s = "VideoDataset("
        for i in range(len(self)):
            s += repr(self[i]) + '\n'
        return s + ')'
    
    def shuffle(self):
        """
            Inplace shuffle
        """
        from random import shuffle
        l = list(zip(self.videoList, self.annotations))
        shuffle(l)
        self.videoList, self.annotations = zip(*l)
    
    def __str__(self):
        s = "VideoDataset("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i]) + '\n'
        else :
            s += str(self[0]) + '\n'*2 + str(self[1]) 
            s += '\n'*2 + ' ... ' + '\n'*2
            s += str(self[-1])
        return s + ')'
    
    def addFolder(self, folder):
        p = join(folderName, "*.mp4")
        for v in glob.glob(p):
            bname = basename(v)
            if not bname in self.videoList:
                annoFile = join(annoPath, splitext(bname)[0])
                annotation = Annotation(annoFile)
                bself.annotations.append(annotation)
                self.videoList.append(abspath(v)) #to avoid 2 identical files

In [22]:
def AverageSequenceSize():
    vd = VideoDataset("/video/Gesture/")
    totLength = [0]*6
    totNumber = [0]*6
    shortestL = 1000
    shortestP = 0
    longestL = 0
    longestP = 0
    for videoName, annotation in vd:
       for seq, gest in annotation:
            totLength[gest] += seq[1]-seq[0]
            totNumber[gest] += 1.0
            if seq[1] - seq[0] < shortestL and gest != 5:
                shortestL = seq[1] - seq[0]
                shortestP = gest
            if seq[1] - seq[0] > longestL and gest != 5:
                longestL = seq[1] - seq[0]
                longestP = gest
    print(shortestL, shortestP)
    print(longestL, longestP)
    print([i/totNumber[j] for j,i in enumerate(totLength)])

In [23]:
AverageSequenceSize()
blalbvla

(3, 4)
(113, 1)
[26.586206896551722, 26.26896551724138, 27.217993079584776, 27.748275862068965, 26.317241379310346, 29.577455919395465]


In [28]:
def cutSequence(srcDir="/video/Gesture", destDir="/video/GestureSequence/"):
    """
        Cut all videos from srcDir into the destDir
        
        Each sequence will go in the subrepository corresponding to the class inside destDir
    """
    import cv2
    import os
    import os.path
    
    videoDataset = VideoDataset(srcDir)
    
    videoNumber = 0
    for video, annotation in videoDataset:
        vCap = cv2.VideoCapture(video)
        cframe = 0
        
        annoNumber = 0
        for (s0, s1), g in annotation:
            while cframe <= s0:
                cframe += 1
            
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            videoName = str(videoNumber)+"-"+str(annoNumber)+".avi"
            videoDir = os.path.join(destDir, str(g))
            if not os.path.isdir(videoDir):
                os.mkdir(videoDir)
            videoName = os.path.join(videoDir, videoName)
            record = cv2.VideoWriter(videoName, fourcc, 30, (int(vCap.get(3)), int(vCap.get(4))))
            
            while cframe <= s1:
                ret, frame = vCap.read()
                if ret:
                    record.write(frame)
                else:
                    return -1
                cframe +=1
            
            
            annoNumber +=1
        videoNumber += 1
    